In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import re
import warnings
warnings.filterwarnings('ignore')
!pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 13.9MB/s 
     |████████████████████████████████| 3.0MB 19.3MB/s 
     |████████████████████████████████| 890kB 30.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=61886e0644ac143fe3e01501ed07dbf916dd7e58104842f2e6abb37414862d16
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# Acquiring preprocessed_dataset
tbs_df = pd.read_csv('/content/drive/My Drive/AAIC Course/Personal case study - StackOverflow/tbs_df.csv')
tbs_df = tbs_df.fillna(' ')

In [ ]:
# defining a function to remove stop_words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.add('would')
stop_words.update([chr(c) for c in range(97, 123)])
# stop_words.remove('no'); stop_words.remove('not'); stop_words.remove('nor')

def stopwrd_removal(sent):
  lst = []
  for wrd in sent.split():
    if wrd not in stop_words:
      lst.append(wrd)
  return " ".join(lst)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Note: BERT can handle stopwords and digits by its own way, hence I am not removing them**

In [ ]:
def text_preprocessor(column):
  """pass any column with Text in it from tbs_df | Note: returns nothing makes inplace changes in tbs_df"""
  # 1. remove html tags, html urls, replace html comparison operators
  text = tbs_df[column].values
  tbs_df[column] = [re.sub('<.*?>', '', i) for i in text]
  tbs_df[column] = tbs_df[column].str.replace('&lt;', '<')\
                                          .str.replace('&gt;', '>')\
                                          .str.replace('&le;', '<=' )\
                                          .str.replace('&ge;', '>=')

  # 2. remove latex i,e., mostly formulas since it's mathematics based dataset
  tbs_df[column] = [re.sub('\$.*?\$', '', i) for i in text]

  # 3. all lowercase 
  tbs_df[column] = tbs_df[column].str.lower()

  # 4. decontractions
  tbs_df[column] = tbs_df[column].str.replace("won't", "will not").str.replace("can\'t", "can not").str.replace("n\'t", " not").str.replace("\'re", " are").str.\
                                                replace("\'s", " is").str.replace("\'d", " would").str.replace("\'ll", " will").str.\
                                                replace("\'t", " not").str.replace("\'ve", " have").str.replace("\'m", " am")

  # 5. remove all special-characters other than alpha-numericals
  tbs_df[column] = [re.sub('\W', ' ', i) for i in text]
  # # remove all digits
  # tbs_df[column] = [re.sub('\d', ' ', i) for i in text]

  # 6. Stop_word removal
  # tbs_df[column] = [stopwrd_removal(i) for i in text]

  # 7. remove all white-space i.e., \n, \t, and extra_spaces
  tbs_df[column] = [re.sub('  +', ' ', i) for i in text]
  tbs_df[column] = tbs_df[column].str.replace("\n", " ").str.replace("\t", " ").str.strip()
  

In [ ]:
# 1. train_test split

tbs_df['Title_1'] = tbs_df['Title']
text_preprocessor('Title_1')
title = tbs_df['Title_1'].values
len(title)

182039

In [ ]:
final_train = title
final_train[:5], tbs_df['Title'].values[:5]

(array(['euler cycles in biconnected components',
        'can two neighbors in a graph be at the same depth in a dfs tree',
        'unique path in a directed graph',
        'what do you call an average that does not include outliers',
        'correcting for outliers in a running average'], dtype=object),
 array(['Euler cycles in biconnected components',
        'Can two neighbors in a graph be at the same depth in a DFS tree?',
        'Unique path in a directed graph',
        'What do you call an average that does not include outliers?',
        'Correcting for outliers in a running average'], dtype=object))

# 1. BERT Embeddings : Using Huggingface

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

# Load pretrained model/tokenizer

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
tokens = bert_tokenizer.batch_encode_plus(final_train, pad_to_max_length = True)
# tokens = bert_tokenizer.batch_encode_plus(final_train, max_length = 50, truncation = True, pad_to_max_length = True)

input_ids  = np.array(tokens['input_ids'])
attn_mask = np.array(tokens['attention_mask'])
token_typ_ids = np.array(tokens['token_type_ids'])
seq_len = input_ids.shape[1]

input_ids.shape, attn_mask.shape, token_typ_ids.shape

((182039, 44), (182039, 44), (182039, 44))

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [ ]:
# 1. defining a BERT model
bert_input_ids = Input(name = 'bert_input_ids', shape = (seq_len,), dtype = 'int64')
bert_attn_mask = Input(name = 'bert_attn_mask', shape = (seq_len,), dtype = 'int64')
bert_token_typ = Input(name = 'bert_token_typ', shape = (seq_len,), dtype = 'int64')

bert_output = bert_model({'input_ids': bert_input_ids, 'attention_mask' : bert_attn_mask, 'token_type_ids': bert_token_typ})

bert_output = bert_output[0][:,0,:]
# bert_output = bert_output[1][:]
# bert_output = tf.concat([bert_output[-1][:,0,:], bert_output[-2][:,0,:], bert_output[-3][:,0,:], bert_output[-4][:,0,:]], axis = -1)[0]

BERT = Model(inputs = [bert_input_ids, bert_attn_mask, bert_token_typ], outputs = bert_output)
BERT.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bert_attn_mask (InputLayer)     [(None, 44)]         0                                            
__________________________________________________________________________________________________
bert_input_ids (InputLayer)     [(None, 44)]         0                                            
__________________________________________________________________________________________________
bert_token_typ (InputLayer)     [(None, 44)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 44, 768), (N 109482240   bert_attn_mask[0][0]             
                                                                 bert_input_ids[0][0]         

In [ ]:
%%time
bert_out = BERT.predict([input_ids, attn_mask, token_typ_ids], batch_size= 128)
print(bert_out.shape)

(182039, 768)
CPU times: user 2min 24s, sys: 49.7 s, total: 3min 13s
Wall time: 15min 56s


In [ ]:
np.save('/content/drive/My Drive/AAIC Course/Personal case study - StackOverflow/bert_train_out.npy', bert_out)